In [91]:
%reset -f

In [92]:
#!/usr/bin/env python3
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import torch
from torch import nn
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy
from torcheval.metrics.functional import binary_accuracy
torch.manual_seed(18)
torch.cuda.is_available()
from FacesDataset import FacesDataset

In [93]:
from DiscriminatorV3 import DiscriminatorV3, ConvBlock

In [94]:
from Generator import Generator

In [95]:
def train(generator, discriminator, optimizer, epochs, loss_fn, device='cuda'):
    target = torch.tensor([0.0]).to(device)
    
    generator.to(device)
    discriminator.to(device)

    generator.eval()
    discriminator.train()

    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    for epoch in (range(epochs)):
        # train the model on the training set
        optimizer.zero_grad()
        x = torch.randn(1, 32).to(device)
        generated_image_tensor = generator.forward(x)
        output = discriminator.forward(generated_image_tensor)
        loss = loss_fn(output.flatten().float(), target)
        loss.backward()
        optimizer.step()
        metric.update(output.flatten(), target)
    print(round(output.item(), 4), round(loss.item(), 4), metric.compute().item())

In [96]:
discriminator = DiscriminatorV3()
discriminator.load_state_dict(torch.load('models/FaceNetV3-Batch-8.pth'))

generator = Generator()

loss_fn = nn.BCELoss()
params = discriminator.parameters()
learning_rate = 3e-4
optimizer = torch.optim.Adam(params, lr=learning_rate)

In [ ]:
# train the discriminator on the generator's output
num_epochs = 100
train(generator=generator, discriminator=discriminator, optimizer=optimizer, loss_fn=loss_fn, epochs=num_epochs)

In [102]:
from torchvision import transforms

device = 'cuda'
x = torch.randn(1, 32).to(device)
generated_image_tensor = generator.forward(x)
print(discriminator.forward(generated_image_tensor))
transforms.ToPILImage()(generated_image_tensor.squeeze(0)).show()

tensor([[2.5863e-05]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [98]:
def train(generator, discriminator, optimizer, epochs, loss_fn, device='cuda'):
    target = torch.tensor([0.0]).to(device)
    
    generator.to(device)
    discriminator.to(device)

    generator.eval()
    discriminator.train()

    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    for epoch in (range(epochs)):
        # train the model on the training set
        optimizer.zero_grad()
        x = torch.randn(1, 32).to(device)
        generated_image_tensor = generator.forward(x)
        output = discriminator.forward(generated_image_tensor)
        loss = loss_fn(output.flatten().float(), target)
        loss.backward()
        optimizer.step()
        metric.update(output.flatten(), target)
    print(round(output.item(), 4), round(loss.item(), 4), metric.compute().item())